# model 입력으로 필요한 데이터만 추출

### 1. 고객리뷰 & 사장답글 길이에 따라 drop
너무 짧거나 긴 고객리뷰 및 사장답글 제외

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("prev2_final_total_data_v2.csv")
len(df)

256030

In [3]:
leng_df = []
for row in df.values:
    try:
        if len(row[6]) < 15 or len(row[6]) > 100 or len(row[7]) < 50 or len(row[7]) > 200:
            continue
        leng_df.append(row)
    except:
        print(row[6], '||', row[7])
len(leng_df)

사진을 못찍었네요ㅠㅠ 잘먹었어요~ || nan


114611

In [4]:
leng_df = pd.DataFrame(leng_df, columns=["업체명","맛","양","배달","주문메뉴","고객id","고객리뷰","사장답글"])
leng_df.to_csv("drop_by_length_v2.csv", encoding="utf-8", index=False)

### 2. 고객리뷰 - 사장답글 사이 유사도에 따라 drop
고객리뷰와 사장답글의 유사도가 0인 것 제외

In [5]:
sim_df = pd.read_csv("../final_all_sim.csv")
sim_df.sample(5)

,업체명,맛,양,배달,주문메뉴,고객id,고객리뷰,사장답글,length,clen,SBERT,tf-idf,glove
60278,던킨-청량리역사점,5,5,5,"\n 핫볼,베이컨스크램블/1,페이머스 글레이즈드/3,밀크티/1,...",손님,배달 빨리 왔어요 포장도 깔끔하고!!\n잘 먹었어요!,저희 매장을 이용해 주셔서 감사합니다\n포장에 항상 신경쓰고 있는데 마음에 들었다니...,69,28,0.76,0.0,0.561989
110241,뚜레쥬르-공릉2동점,5,5,5,\n 라즈베리 웨이퍼 샌드/1\n,zz**님,선물용으로 구입했습니다.,"고객님, 주문 감사합니다. 맛있는 제품으로 주문해주셨네요 ㅎㅎ. 만족하셨으면 좋겠습...",73,13,0.30,0.0,0.774601
122577,파리바게뜨-고대역점,4,4,5,"\n 명품 잡채고로케/1,모닝토스트/1,리코타치즈&치킨샐러드/1...",km**님,행사로 저렴하게 구매.\n맛 있습니다.\n강추합니다,리코다샐러드소스에 모닝토스트\n결대로 쫘악~ 찢어서 찍어드시면\n순삭하실수있어요~...,112,26,0.53,0.0,0.515097
124877,배스킨라빈스-광나루역점,5,5,5,"\n 쿼터/1(제품 밀봉 배달 전용 패키지(선택 안함),다회용 ...",an**님,배달도최고 상태도 최고\n짱 입니다,저희 대행사 기사님들이 빠르게 배송해드리고 있습니다 \n🥰🥰\n맛있는 아이스크림을 ...,86,18,0.71,0.0,0.685987
169743,크리스피크림도넛-메세나폴리스점,5,5,5,\n 어쏘티드 더즌/1\n,ne**님,맛있게 잘 먹엏오요,안녕하세요! 이용해주셔서 감사합니다. \n앞으로 맛있는 도넛 배달하도록 하겠습니다....,56,10,0.34,0.0,0.000000


In [6]:
sim_df = sim_df.drop(["업체명", "주문메뉴", "고객id"], axis=1)

In [7]:
sim_df.iloc[0, 4]

'매장 시그니쳐 메뉴인 육회비빔밥\n메뉴 맛있게 드셨다니 뿌듯하네요~\n\n언제 주문하시더라도 음식이 식거나\n음료가 녹지 않도록 빠른 배달로\n주문에 보답하겠습니다~'

### [5] (사장답글) length column 정렬

[min = 1, max = 306]

- 길이가 1인 경우는 모델이 학습으로 삼기에는 성의가 없음

In [8]:
length_df = pd.DataFrame(sorted(sim_df.values, key=lambda x: x[5]))

In [9]:
length_df.head(26)

,0,1,2,3,4,5,6,7,8,9
0,1,3,1,"1시간반지나서야 배달이왔네요..\n여러가지 상황상 늦을순 있지만, 미리연락이라도 있...",감,1,60,0.29,0.0,0.513710
1,5,5,5,굿,훗,1,1,0.79,0.0,0.000786
2,5,5,5,잘 먹었어요~~~~,휴,1,10,0.79,0.0,0.000000
3,4,4,4,빠른 배달 감사합니다,ㅠ,1,12,0.54,0.0,0.000000
4,5,5,5,저렴한가격에 종류별로 맛나게 먹었네요ㅎ\n다음에 또 주문할께요,넵,1,33,0.85,0.0,-0.171188
5,4,3,3,빠바빠바빠바빠바입니다,😍,1,11,0.83,0.0,0.000000
6,5,5,5,맛있게 잘 먹었어요~,🥰,1,11,0.86,0.0,0.000000
7,5,5,별점X,행사때 자주 구입해요~,😊,1,12,0.78,0.0,0.000000
8,5,5,5,사진엔 없지만 ㅜ 서비스로 쫀드기도 보내주시고 맛있게 먹었습니다~~!,♡,1,38,0.67,0.0,0.000000
9,2,2,5,오늘...... 6월 21일........\n유통기한 6/22일..... 짬처리...?,-,1,47,0.52,0.0,0.000000


In [10]:
length_df.tail(3)

,0,1,2,3,4,5,6,7,8,9
256027,5,5,5,구땡이 스프 진짜 중독 ㅠㅠㅠ\n샐러드는 말해 뭐해용~\n바질소스 먹물빵으로 바꿔 ...,안녕하세요!! 구땡이야채집 주문 감사 드립니다!!\n아이고 ㅠㅠ 이번 주문에도 맛있...,304,84,0.16,0.023948,0.752141
256028,5,5,5,오늘 배송도 완전빠르고 ㅎㅎ\n음료서비스까지 최고입니다^^\n친절하게 배달해주이고...,ch**님 \n우리우리 단골 고객님~그렇쥬? 전 다 알아여...다 기억할테얌 ^.^...,304,65,0.49,0.000000,0.723068
256029,5,5,5,"새로나온 베이컨샌드위치 시켰는데, 맛있어요!\n샐러드는 오랜만에 주문했는데, 패키지...",안녕하세요~ 고객님~\n베이컨 샌드위치~ 맛있는 재료들만 들어있어서~ 맛없맛 샌드위...,306,110,0.71,0.013951,0.878693


### [6] clen column 정렬

[min = 1, max = 316]

- 고객리뷰가 심플할 경우, 낮은 점수

In [11]:
clen_df = pd.DataFrame(sorted(sim_df.values, key=lambda x: x[6]))

In [12]:
clen_df.head(12)

,0,1,2,3,4,5,6,7,8,9
0,5,5,5,.,주문과 함께 별다섯개 소중한\n리뷰 남겨주셔서 감사합니다~\n\n언제 주문하셔도 맛...,107,1,0.53,0.0,0.000000
1,2,5,4,.,너무 고맙습니다,8,1,0.82,0.0,0.000000
2,4,5,4,.,맛있게 드셨나요!,9,1,0.45,0.0,0.000000
3,5,5,5,굿,안녕하세요~🙋‍♂️ 고객님~\n오늘은 훈제 스타일로다가~\n베이컨 🥓 & 햄 🥓 ...,194,1,0.55,0.0,0.471322
4,5,5,5,👍,굿~!!!,5,1,0.85,0.0,0.000000
5,5,5,5,굳,굳굳~ 만족스럽게 드셨다니 넘나 행벅합니당. 감사합니~다,31,1,0.67,0.0,0.000000
6,5,5,5,ㆍ,힘든시기에 주문해주시고 리뷰남겨주셔서감사합니다 👍앞으로도 정성껏맛있게보네드릴께요!...,57,1,0.75,0.0,0.000000
7,5,5,5,ㆍ,별다른 말씀 없으셨지만 고객님 마음알거 같습니다^^감사드립니다\n다음주문시에도 만...,81,1,0.62,0.0,0.000000
8,5,5,4,굿,감사합니다.,6,1,0.73,0.0,0.000000
9,5,5,5,👍,고객님 나중에 연유라떼랑 아이스박스 드셔보세요 진짜로 넘무 맛있어요🥲,38,1,0.70,0.0,0.000000


In [13]:
clen_df.tail(4)

,0,1,2,3,4,5,6,7,8,9
256026,5,5,5,아니 베이글이 이렇게 맛있다구요??!!!!!!!!!👏👏👏👏👏❣️❣️❣️❣️❣️\n베...,"맛있게 드셔주셔서 감사합니다 ^^ \n크랜베리,초코베이글이 다른 베이글에 비해 쫀득...",91,308,0.62,0.074486,0.757231
256027,5,5,5,제가 첫리뷰네용🙆🏻‍♀️🙆🏻‍♀️\n사진은 친구가 마신 아아인데 제가 주문한 아바라...,"고객님, 첫 리뷰 감사합니다. 저의 최애 음료도 아바라여서 제조법을 정할 때 고민이...",138,309,0.69,0.022274,0.906994
256028,5,5,5,일단 배달 완전 빨리 왔어요!! 빽다방 아바라는 모든 카페 통틀어 제가 가장 사랑하...,안녕하세요 고객님\n어머머.. 전 이런 리뷰 볼 때마다 고객님들께 사랑 받는 기분이...,201,312,0.32,0.000000,0.887059
256029,5,5,5,점심에 먹어도 저녁에 또 먹고 싶은 맛...\n최근에 고민하다가 처음 시켜먹었는데 ...,정성어린 긴 리뷰 감사합니다😭 나름 정성들여 만들고 있긴한데 너무 칭찬해주셔서 몸둘...,128,316,0.39,0.000000,0.779113


### [7] SBERT column 정렬

[min = -0.15, max = 1.00]

- 낮은 점수 임에도 고객과 소통하는 답글에 해당
    - 반대되는 단어 & 함축적인 의미 풀이를 잘못 예측한다고 판단하는 걸로 분석
        (ex. 눅눅함<->바삭함, 매일 먹는다<->매일 이용한다)


- 높은 점수일 경우 고객리뷰와 거의 동일한 형태



In [14]:
sbert_df = pd.DataFrame(sorted(sim_df.values, key=lambda x: x[7]))

In [15]:
sbert_df.head(8)

,0,1,2,3,4,5,6,7,8,9
0,5,5,5,맛있게 잘먹었습니다.,감사합니다. 즐거운 휴일되시고\n다음 행사때도 이용해 주세요.^^,35,11,-0.15,0.000000,0.000000
1,5,5,5,맛있게 먹었습니다 현장에서 먹는게 아니라 어쩔 수 없이 좀 눅눅한 감이 있지만 만족...,감사합니다.^^\n좀더 바삭하게 준비 하겠습니다.^^,28,52,-0.14,0.000000,0.408430
2,5,5,별점X,직원분친절하고 편하게 잘사서 만족해요.,열심히 노력중입니다.\n고맙습니다^^,19,21,-0.14,0.000000,0.230287
3,5,5,5,항상 세트 시켜먹는데 맛있어요.,안녕하세요^^\n감사합니다!!\n더욱더 신경써서 준비하겠습니다 \n날씨가 많이 쌀쌀...,58,17,-0.13,0.000000,0.398437
4,4,5,별점X,다이어트 때문에 매일 먹고 있어요 괜찮아요.,매일 이용해주시는 거네요.너무 감사합니다^^,24,24,-0.13,0.101631,0.847548
5,5,5,별점X,픽업했는데 직원분도 친절하고 포장도 깔끔하고 하나씩 꺼내먹기 편하게 되어있어 넘 좋...,이용해주셔서 감사합니다. \n던킨과 달콤한 하루가 되시길 바랍니다.^^,38,50,-0.12,0.000000,0.593209
6,5,5,별점X,집가까워 자주애용해요.\n샐러드많이들어있어서좋아요,카페 전문 기사가 재료 아끼지않고 넣고 있답니다.^^,29,26,-0.12,0.000000,0.626812
7,5,5,별점X,매번 잘 먹습니다.,매번 주문해 주셔서 감사합니다.^^,19,10,-0.12,0.220288,0.738611


In [16]:
sbert_df.tail(4)

,0,1,2,3,4,5,6,7,8,9
256026,5,5,5,맛있어요~~\n빠른 배달 감사합니당,맛있게 드셔서 감사합니당~,14,18,0.99,0.170776,0.000000
256027,5,5,5,감사합니다~~~^^,감사합니다~~~^^,10,10,1.00,1.000000,0.000000
256028,5,5,5,👍👍👍👍👍👍👍👍👍👍👍,👍👍👍👍👍👍👍👍👍👍👍,11,11,1.00,0.000000,0.000000
256029,5,5,5,든킨드나스 존맛탱이구요~~~~,든킨드나스 존맛탱탱볼~~~~,15,16,1.00,0.336097,0.455181


### [8] tf-idf column 정렬

[min = 0.0, max = 1.0]

- 높은 점수의 경우 고객리뷰와 비슷하면서도 사장님으로서의 답글 

- 낮은 점수의 경우 메뉴/별점에 대한 정보 언급

In [17]:
tfidf_df = pd.DataFrame(sorted(sim_df.values, key=lambda x: x[8]))

In [18]:
tfidf_df.head(8)

,0,1,2,3,4,5,6,7,8,9
0,5,5,5,맛있어요 친절하고요,매장 시그니쳐 메뉴인 육회비빔밥\n메뉴 맛있게 드셨다니 뿌듯하네요~\n\n언제 주문...,87,10,0.68,0.0,0.000000
1,5,5,5,☆☆☆☆☆☆☆☆☆☆,"미식화의 간단한 간식거리와 \n음료 한 잔, 코젤다크 생맥주를 주문해주셨네요~\n주...",123,10,0.29,0.0,0.000000
2,5,5,5,매번 먹을때마다 만족합니다~\n번창하시길~~,드실 때 마다 만족하신다니..\n너무 감사합니다~\n\n언제 주문하시더라도 음식이 ...,77,23,0.53,0.0,0.649639
3,5,5,5,잘먹었습니다 수고하세요,별 5개 리뷰를 달아주셔서 감사합니다~\n\n언제 주문하시더라도 음식이 식거나\n음...,72,12,0.69,0.0,0.252730
4,5,5,5,맛있게잘먹었습니다.,주문과 함께 별 다섯개 리뷰와 \n소중한 사진까지 남겨주셔서\n너무 너무 감사합니다...,112,10,0.55,0.0,0.000000
5,5,5,5,맛있어요 배달도 빠랐어요~,주문과 함께 별 다섯개 리뷰와 \n소중한 사진까지 남겨주셔서\n너무 너무 감사합니다...,132,14,0.50,0.0,0.567301
6,4,5,5,사진에 옥수수콘 있어서 주문한건데 없어서 당황했어욤..ㅠㅠ 전화했더니 콘이 떨어졌다...,고객님 주문에 만족을 못드린점\n무척이나 죄송스럽습니다ㅜㅜ\n\n하필 그날 옥수수 ...,223,95,0.74,0.0,0.791215
7,2,3,5,"전보다 양 줄은건 후기통해 알았지만,\n양념도 너무줄었어요. 너무 싱거웠어요ㅠ",우선 고객님 주문에 만족을 못드려\n무척이나 죄송스럽습니다ㅜㅜ\n\n육회양은 이전 ...,210,42,0.47,0.0,0.846020


In [19]:
tfidf_df.tail(13)

,0,1,2,3,4,5,6,7,8,9
256017,3,4,5,솔직한 리뷰 솔직한 리뷰,솔직한 리뷰 감사합니다❤ \n솔직한 리뷰 감사합니다❤,28,13,0.87,0.709297,1.000000
256018,5,5,별점X,샐러드 맛집~♡\n샐러드 맛집~♡,샐러드 맛집 감사합니다~~!,15,17,0.77,0.709297,1.000000
256019,5,5,5,감사합니다.\n추천합니다!\n엄지척!,감사합니다!\n엄지척!!!!,14,18,0.75,0.709297,0.895574
256020,5,5,5,새해 복 많이 받으세요,새해 복 많이 많이 받으세요. 매일매일 행복하세요.,28,12,0.92,0.732105,0.851933
256021,5,5,5,고객님의 리뷰는 사랑입니다.,그럼여\n고객님의 리뷰는 사랑입니다🤍🤍,20,15,0.59,0.776515,1.000000
256022,5,5,5,완전 역시 변하지않는 그 풍미로운맛👍,완전 역시 변하지않는 만점리뷰의 그 풍미로운맛👍,26,20,0.97,0.818180,0.931077
256023,5,5,별점X,감사합니다 감사합니다,저희가 더 감사합니다 감사합니다😁,18,11,0.92,0.818180,0.000000
256024,5,5,5,"우와 너무 너무 맛있었습니다, 감사합니다!!! 😍😍😍",우와 너무 너무 감사합니다!!ㅎ,17,29,0.55,0.867364,0.000000
256025,5,5,5,감사합니다 감사합니다 감사합니다,저도 감사합니다 감사합니다 감사합니다^^!!,24,17,0.76,0.905550,0.000000
256026,5,5,별점X,감사합니다 감사합니다 감사합니다,제가더 감사합니다 감사합니다 감사합니다,21,17,0.98,0.905550,0.000000


### [9] glove column 정렬

[min = -0.6, max = 1.0]

In [20]:
glove_df = pd.DataFrame(sorted(sim_df.values, key=lambda x: x[9]))

In [21]:
glove_df.head(6)

,0,1,2,3,4,5,6,7,8,9
0,1,1,1,두시간반 넘게 걸려서 받았네요,너무 늦게 받게 해드려 죄송합니다\n라이더가 늦게 잡힌것 같습니다\n이런일 없도록 ...,58,16,0.73,0.000000,-0.605853
1,2,2,1,너무늦는다 어케 한시간반을기다리게하냐,wl**님 안녕하세요~\n최대한 빠르게 보내드리도 싶지만 배달기사 분들배차\n잡기가...,82,20,0.57,0.000000,-0.529161
2,5,5,5,카페라떼 존맛탱 꺄악,"qh**님 주문해 주셔서 감사합니다.\n주문해 주신 것 만도 감사한데, 이렇게 🌟 ...",102,11,0.76,0.000000,-0.494597
3,5,5,5,너무 맛있어요 치즈볼까지 다 맛있어요 감사합니다!,"cm**님, 저희 SKY 31을 이용해주셔서 감사합니다~만족스런 식사되셧다니 기쁘네...",136,27,0.60,0.079348,-0.486023
4,3,3,3,흑임자가 연해졌어요..,아넵~ 찾아주셔서 정말 감사합니당(__)! 들어가는 건 항상 동일하게 들어갑니다~!...,82,12,0.66,0.000000,-0.476069
5,5,5,5,이집 잘하네....,감사합니다\n항상 잘하는 집으로 인정받기위해 노력할게요\n행복한 일요일저녁 보내세요~♡,46,10,0.76,0.000000,-0.469159


In [22]:
glove_df.tail(4)

,0,1,2,3,4,5,6,7,8,9
256026,5,5,5,맛있어요 추천합니다.,맛있게드시고 추천까지 해주셔서 감사합니다♡,23,11,0.41,0.000000,1.0
256027,5,5,별점X,쿠키도 잘 먹겠습니다 감사해용,쿠키 맛잇쥬..⭐️감사합니다!,16,16,0.95,0.000000,1.0
256028,5,5,별점X,잘 먹었습니다 쿠키 맛있어여,쿠키 맛있게 드셔주셔서 감사합니당>_<♡,22,15,0.83,0.170776,1.0
256029,5,5,5,오랜만에 먹었어요 잘먹었습니다^^,오랜만에 드셔도 맛있는 타타타! 감사합니다!,24,18,0.58,0.150640,1.0
